## Assignment 3 (foma)

### Part 1: Regular expressions (20 pts)

In this assignment you will have three parts. In the first part, you will have to write a simple `foma` file where you define four regular expressions. When you create an automaton or a tranducer, foma shows the information about it. In this exercise I include the size of the transducers, so that you can do a sanity check. These are the regexps that you have to write:  

  1. Any word that ends in `t` and begins with `p`: `{pat, papat, ...}`. `[Size of answer: 3 states, 7 arcs, Cyclic.]`

  2. Any word that ends in `t` or begins with `p`, but NOT both, as in `pat`. `[Size of answer: 5 states, 15 arcs, Cyclic.]`
  
  3. Any word that doesn't contain both `p` and `t`: `{papa, tata, baba,...}`, `NOT: pata, tapa, etc.` That is, any word in the language **may or may not contain `p` and `t` but cannot contain both**. `[Size of answer: 3 states, 7 arcs, Cyclic.]`
  
  4. Any word that may contain a `p`, but only if followed by `t`: `{a, b, apta, ptapt, ...} NOT e.g.: pap` since `p` isn't followed by `t`. `[Size of answer: 2 states, 4 arcs, Cyclic.]`

The answers should look like this:

`def One    YOUR REGEX for 1. HERE ;
def Two    YOUR REGEX for 2. HERE ;
def Three  YOUR REGEX for 3. HERE ;
def Four   YOUR REGEX for 4. HERE ;`

### Part 2: Noisy channel (20 pts)

Remember the old T9-input for cellular phones? This was a system where, on a phone keypad, letter symbols would correspond to numbers in a systematic way, like so:

```
+-------------+
| 1  | 2 |  3 |
|    |abc| def|
+-------------+
| 4  | 5 |  6 |
|ghi |jkl| mno|
+-------------+
| 7  | 8 |  9 |
|pqrs|tuv|wxyz|
+-------------+
|      0      |
|   (space)   |
+-------------+
```

If you typed sequences of numbers, the system would try to guess what you intended to write (i.e. decode your message into letters).

However, since there are more letters than numbers, if you type out a string as numbers it could correspond to several valid words in English. For example, if you typed `269`, you could have intended the word any or bow or box or boy or cow, since these all would correspond to that number sequence.

In this assignment, you will construct a transducer that will decode a word or a string of words (with spaces in between) expressed as a long sequence of numbers, and produce all the possible sequences of words that could be found in a lexicon of English. The lexicon itself is already given to you in the form of a wordlist ([engwords.txt](https://absalon.ku.dk/files/1717225/download?download_frd=1)).

As a first step, you should read this wordlist into foma and give it a name, like so:

```read text engwords.txt
def Words;```

The read text command simply reads a file with one word per line and creates an automaton (or repeating transducer as we like to think of it) that accepts all those words.

Now, the task is to define a transducer T that maps any number sequences to all possible letter sequences as defined by the keyboard above. However, such a transducer by itself is useless since it has no knowledge of word sequences. Let's say you defined such a transducer T in foma and the tested it like so:

```foma[0]: regex T;
1.5 kB. 1 state, 27 arcs, Cyclic.
foma[1]: down
apply down> 20269
a amw
a amx
a amy
a amz
...```


you will find that this indeed outputs all the possible combinations of letters that could have been intended. But we also want to combine this with the lexicon contained in Words to "filter out" sequences that are not valid words of English.

The way to do this is to first define your mapping transducer `T` and then compose it with the lexicon Words, like so:

`regex T .o. [Words " "]* Words;`


Note how the second part of the composition is defined: it essentially is a repeating transducer that allows one or more words from the lexicon with spaces in between. If you have defined `T` correctly, the system should now behave as follows:

```foma[1]: down
apply down> 20269
a any
a bow
a box
a boy
a cow```

and, in the other direction:

```foma[1]: up
apply up> two words
896096737```

Having defined `T` in the correct fashion, your task is to decode this secret message:

`26374226047020837903433428580526482430649330948403645474`

To recap: all you need to do is define the transducer `T` and combine it with the lexicon, as shown above, and then do:

```foma[1]: down
apply down> 26374226047020837903433428580526482430649330948403645474```

And you should have the answer.

The solution you hand in should look like this:

```read text engwords.txt
def Words;
def T  YOUR REGEX FOR T HERE ;
regex T .o. [Words " "]* Words;
down 26374226047020837903433428580526482430649330948403645474```

### Part 3: Morphological parsing (40 pts)

In this assignment, you are given the task of completing the morphophonological component of a toy foma grammar of English. A lexicon has already been built for you, and is available as a [lexc-file](https://absalon.ku.dk/courses/19985/files/1717224/download?download_frd=1), which you can compile, and your job is to fill in the morphophonological rules. The first thing you should do is read in the lexc-file called able.lexc and look at the input/output pairs:

```read lexc able.lexc
pairs```

You'll notice that many of the surface forms are not correct and require adjustment by morphophonological rules. For example, you have pairs such as:



`fly+able+ity[N][Pl]   fly+able+ity+s`

where (disregarding the +-symbols which separate morphemes) the correct surface form should be flyabilities.

The base lexicon consists of only five verbs: `think`, `excuse`, `run`, `fly`, and `watch`. These verbs (tagged `[V]`) can be inflected in the first `[1p]`, second `[2p]`, and third `[3p]` person (present tense). Also, adding a suffix able changes the verb into an adjective (tag `[A]`). Adding ity after the able produces a noun `[N]`. Nouns can be inflected in singular `[Sg]` and plural `[Pl]`.

You don't have to worry about the tags (or the input side) at all, since this part of the grammar has already been designed for you. Your job is to fix the morphophonology with some rewrite rules so that the correct surface forms are generated.

The morphophonological component requires you to do the following things with rewrite rules, composed together:

 * Convert the sequence able+ity into ability, always.
 * Turn y into ie before +s, e.g. fly+s > flie+s
 * Turn n into nn before +V where V is any vowel. (note that this should happen with all consonants, i.e. they should double, but we only have one word ending in a consonant here, run, so it's enough to just double the n).
 * Delete vowels when they occur before + and another vowel: excuse+able > excus+able.
 * Insert an e between a sibilant on the left and +s on the right: watch+s > watche+s. The sibilants are consonant sequences sh,ch,s,x,z, although only ch occurs in this data set. NOTE: insertion rules should begin with `[..] -> RHS`, NOT `0 -> RHS`.
 * Last, you want to delete all +-symbols (already included in the skeleton code) below.

In other words, you should expect to write five separate rewrite rules, compose them in the right order, plus the last Cleanup rule which simply removes +-symbols.

In the formulation of these rules, you can look at the [Morphological Analysis Tutorial](https://fomafst.github.io/morphtut.html). Chapter 3 of Jurafsky and Martin (2009) also discusses these phenomena and may be a good reference. Reference at the bottom.

The last line in the skeleton code simply prints out all the underlying/surface pairs. They should come out as follows, i.e. this is the result you want in the end:

```think+able[A]            thinkable
think+able+ity[N][Pl]    thinkabilities
think+able+ity[N][Sg]    thinkability
think[V][3p][Sg]         thinks
think[V][2p][Sg]         think
think[V][1p][Sg]         think
excuse[V][3p][Sg]        excuses
excuse[V][2p][Sg]        excuse
excuse[V][1p][Sg]        excuse
excuse+able[A]           excusable
excuse+able+ity[N][Pl]   excusabilities
excuse+able+ity[N][Sg]   excusability
watch+able[A]            watchable
watch+able+ity[N][Pl]    watchabilities
watch+able+ity[N][Sg]    watchability
watch[V][3p][Sg]         watches
watch[V][2p][Sg]         watch
watch[V][1p][Sg]         watch
fly[V][3p][Sg]           flies
fly+able[A]              flyable
fly+able+ity[N][Pl]      flyabilities
fly+able+ity[N][Sg]      flyability
fly[V][2p][Sg]           fly
fly[V][1p][Sg]           fly
run+able[A]              runnable
run+able+ity[N][Pl]      runnabilities
run+able+ity[N][Sg]      runnability
run[V][3p][Sg]           runs
run[V][2p][Sg]           run
run[V][1p][Sg]           run```

Your solution should look like this:

```
read lexc able.lexc  # Read the pre-defined lexicon
def Lexicon;         # Label this transducer "Lexicon"

def YourRule1 [ADD STUFF HERE] ;
def YourRule2 [ADD STUFF HERE] ;
def YourRule3 [ADD STUFF HERE] ;
def YourRule4 [ADD STUFF HERE] ;
def YourRule5 [ADD STUFF HERE] ;
def Cleanup "+" -> 0;

regex Lexicon .o. YourRule1 .o. YourRule2 .o. YourRule3 .o. YourRule4 .o. YourRule5 .o. Cleanup;
pairs
```

and the pairs printed out should correspond to the above correct input/output pairs.

### Format of the files:

The answer to these questions should be written in three separate foma files that should have the following name: `UCPH-USERNAME-partX.source`. Then, if your UCPH name is abc123, your files name for part 1 should be `abc123-part1.source`.

Once that you have three separate files for each exercise, please compress them using `zip` or `tar`. For example, our friend abc123 should execute the following command:

`tar cvzf acb123.tgz abc123-part1.source abc123-part2.source abc123-part3.source`

Or, this one:

`zip acb123.zip abc123-part1.source abc123-part2.source abc123-part3.source`

And upload the resulting file.

Reference:

Jurafsky, D., & Martin, James H. (2009). *Speech and language processing, an introduction to natural language processing, computational linguistics, and speech recognition* (2.nd ed., Prentice Hall Series in artificial intelligence). Upper Saddle River, N.J: Prentice Hall.